In [1]:
from pygtfcode import Config, State

# Instantiate config object and set some parameters

In [2]:
config = Config()

In [3]:
config

Config(
  io=IOParams(model_no=0, base_dir='/Users/yaronetokayer/YaleDrive/Research/SIDM/pygtfcode/tests', model_dir='Model000', overwrite=True),
  grid=GridParams(rmin=0.001, rmax=100.0, Ngrid=300),
  init=NFWParams(Mvir=3000000000.0, cvir=20.0, z=0.0),
  sim=SimParams(sigma_m=10.0, a=2.256758, b=1.38, c=0.75),
  prec=PrecisionParams(eps_du=0.01, eps_dr=0.01, eps_dt=0.01, max_iter_du=10, max_iter_revir=50)
)

In [4]:
config.init = "abg"

In [5]:
config.grid.rmax = 200
config.grid.Ngrid = 200

# Instantiate State

In [6]:
state = State(config)

In [7]:
state

State(
{ 'char': CharParams(r_s=0.0017486734001055774, fc=None, chi=1.1251590902413837, m_s=3808985167.4173994, sigma0=1.0088300038437138e-14, t0=1192949823517890.0, v0=96.77300230168737, rho_s=5.668567284342425e+16),
  'r': array([0.00000000e+00, 1.00000000e-03, 1.06325722e-03, 1.13051592e-03,
       1.20202922e-03, 1.27806625e-03, 1.35891317e-03, 1.44487425e-03,
       1.53627298e-03, 1.63345334e-03, 1.73678106e-03, 1.84664501e-03,
       1.96345865e-03, 2.08766159e-03, 2.21972126e-03, 2.36013466e-03,
       2.50943023e-03, 2.66816982e-03, 2.83695083e-03, 3.01640846e-03,
       3.20721808e-03, 3.41009779e-03, 3.62581111e-03, 3.85516985e-03,
       4.09903719e-03, 4.35833090e-03, 4.63402681e-03, 4.92716248e-03,
       5.23884109e-03, 5.57023563e-03, 5.92259327e-03, 6.29724008e-03,
       6.69558599e-03, 7.11913017e-03, 7.56946658e-03, 8.04829001e-03,
       8.55740249e-03, 9.09872001e-03, 9.67427977e-03, 1.02862478e-02,
       1.09369273e-02, 1.16287670e-02, 1.23643705e-02, 1.31465062